Import Packages (uncomment install lines if necessary)

In [6]:
# pip install folium
# pip install pulp
# pip install openpyxl
# pip install geopandas
# pip install kaleido
# python -m pip install global_land_mask
#conda install -c conda-forge pulp
import kaleido
kaleido.__version__ #0.2.1
import pulp 
import numpy as np
import pandas as pd
import math
import json
import geopandas as gpd
import folium
import plotly.express as px
import plotly.graph_objects as go
import requests
from global_land_mask import globe
import time
import constants # constants.py

Create Facilities.xlsx which is a grid over Europe for all candidate positions of facilities

In [13]:
# Generate the grid coordinates
grid = []
for lat in range(int(constants.min_lat), int(constants.max_lat) + 1):
    for lon in range(int(constants.min_lon), int(constants.max_lon) + 1):
        if globe.is_land(lat, lon):  # Check if coordinates are on land and include if yes
            for i in range(int(constants.grid_spacing)):
                for j in range(int(constants.grid_spacing)):
                    point_lat = lat + i / constants.grid_spacing
                    point_lon = lon + j / constants.grid_spacing
                    params = { 
                        'key': constants.api_key_opencage,
                        'q': f"{point_lat},{point_lon}",
                        'no_annotations': 1
                    } # API to check the country the coordinates are in
                    response = requests.get(constants.base_url_opencage, params=params).json()
                    if 'results' in response and len(response['results']) > 0:
                        country = response['results'][0]['components']['country']
                        grid.append((point_lat, point_lon, country))

# Create a DataFrame with the grid coordinates and country information
df = pd.DataFrame(grid, columns=['Latitude', 'Longitude', 'Country'])

# Reorder the columns
df = df[['Country', 'Latitude', 'Longitude']]

# Add an index column for the Excel file
df = df.reset_index()

# Rename the 'index' column
df = df.rename(columns={'index': 'Name'})

# Replace name of countries so they match the Fixed Cost df
df['Country'] = df['Country'].replace('Czechia','Czech Republic')

# Add a prefix "N" to the Facility name number
df['Name'] = 'N' + df['Name'].astype(str)
df

,Name,Country,Latitude,Longitude


In [17]:
lat = 57.695641429965946
lon = 11.968428704531103

# Return the country of the coordinates
params = { 
    'key': constants.api_key_opencage,
    'q': f"{lat},{lon}",
    'no_annotations': 1
}
response = requests.get(constants.base_url_opencage, params=params).json()
response


{'documentation': 'https://opencagedata.com/api',
 'licenses': [{'name': 'see attribution guide',
   'url': 'https://opencagedata.com/credits'}],
 'results': [],
 'status': {'code': 401, 'message': 'invalid API key'},
 'stay_informed': {'blog': 'https://blog.opencagedata.com',
  'mastodon': 'https://en.osm.town/@opencage'},
 'thanks': 'For using an OpenCage API',
 'timestamp': {'created_http': 'Mon, 25 Sep 2023 10:21:56 GMT',
  'created_unix': 1695637316},
 'total_results': 0}

In [18]:
constants.api_key_opencage

'your_opencage_api'